In [2]:
# Upload fine-tuning files

import os
from openai import AzureOpenAI

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://llm-ft.openai.azure.com/"
os.environ["AZURE_OPENAI_KEY"] = "799f094146ba4a39ae6a59822297e155"

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"  # This API version or later is required to access fine-tuning for turbo/babbage-002/davinci-002
)

In [3]:
training_file_name = './train_data.jsonl'
validation_file_name = './validate_data.jsonl'

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

In [5]:
print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)
training_file_id = "file-2025cdc6f7e048e8bbf76c9d5dd64c55"
validation_file_id = "file-4442c796062b4594bba16d9e80bad581"

Training file ID: file-2025cdc6f7e048e8bbf76c9d5dd64c55
Validation file ID: file-4442c796062b4594bba16d9e80bad581


In [6]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-1106" # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters. 
)

job_id = response.id

In [7]:
# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.
# e51ca50072d046dfa0ed414b874a97d2
print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))

Job ID: ftjob-2f061e1410524dd0b9b984c5d521fef8
Status: ftjob-2f061e1410524dd0b9b984c5d521fef8
{
  "id": "ftjob-2f061e1410524dd0b9b984c5d521fef8",
  "created_at": 1710048269,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": 2,
    "batch_size": 2,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-1106",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-2025cdc6f7e048e8bbf76c9d5dd64c55",
  "validation_file": "file-4442c796062b4594bba16d9e80bad581"
}


In [8]:
response = client.fine_tuning.jobs.retrieve("ftjob-2f061e1410524dd0b9b984c5d521fef8")

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-2f061e1410524dd0b9b984c5d521fef8
Status: succeeded
{
  "id": "ftjob-2f061e1410524dd0b9b984c5d521fef8",
  "created_at": 1710048269,
  "error": null,
  "fine_tuned_model": "gpt-35-turbo-1106.ft-2f061e1410524dd0b9b984c5d521fef8",
  "finished_at": 1710051914,
  "hyperparameters": {
    "n_epochs": 2,
    "batch_size": 2,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-1106",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": [
    "file-4606db28093e45c0928dcce66c792145"
  ],
  "status": "succeeded",
  "trained_tokens": 2491790,
  "training_file": "file-2025cdc6f7e048e8bbf76c9d5dd64c55",
  "validation_file": "file-4442c796062b4594bba16d9e80bad581"
}


In [4]:
response = client.fine_tuning.jobs.retrieve("ftjob-2f061e1410524dd0b9b984c5d521fef8")
if response.status == 'succeeded':
    result_file_id = response.result_files[0]

retrieve = client.files.retrieve(result_file_id)

# Download the result file.
print(f'Downloading result file: {result_file_id}')

with open(retrieve.filename, "wb") as file:
    result = client.files.content(result_file_id).read()
    file.write(result)